# TypeScript Schema Handler - LLM-Optimized Schema Documentation

The TypeScript schema handler converts JSON Schema to TypeScript-style notation for optimal LLM comprehension. This format is used in khivemcp and lionherd for documenting MCP tool schemas.

**Core Features:**
- **Type Mapping**: JSON Schema types → TypeScript-like types (`int`, `float`, `bool`, `string`)
- **Enum Unions**: Enum values as literal unions (`"option1" | "option2"`)
- **Optional Fields**: `?` suffix for optional fields with default values
- **Default Values**: Inline default value display
- **Descriptions**: Inline field documentation
- **Arrays**: TypeScript array notation (`type[]`)
- **Nested Objects**: Proper indentation for readability

In [1]:
from lionherd_core.libs.schema_handlers._typescript import (
    _format_enum_union,
    _type_map,
    typescript_schema,
)

## 1. Basic Type Mapping

JSON Schema types are mapped to TypeScript-like types optimized for LLM understanding.

In [2]:
# Type mapping
print(f"string → {_type_map('string')}")
print(f"integer → {_type_map('integer')}")
print(f"number → {_type_map('number')}")
print(f"boolean → {_type_map('boolean')}")
print(f"array → {_type_map('array')}")
print(f"object → {_type_map('object')}")
print(f"null → {_type_map('null')}")

string → string
integer → int
number → float
boolean → bool
array → array
object → object
null → null


## 2. Simple Schema Conversion

Convert a basic JSON Schema with required and optional fields.

In [3]:
simple_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "User's full name"},
        "age": {"type": "integer", "description": "User's age in years"},
        "email": {"type": "string", "description": "Email address"},
    },
    "required": ["name", "age"],
}

result = typescript_schema(simple_schema)
print(result)
print("\nNote: 'email' has '?' suffix (optional), 'name' and 'age' don't (required)")

name: string - User's full name
age: int - User's age in years
email?: string - Email address

Note: 'email' has '?' suffix (optional), 'name' and 'age' don't (required)


## 3. Enum Values as Literal Unions

Enums are formatted as TypeScript literal unions for clear value constraints.

In [4]:
# Format enum values
status_enum = ["pending", "active", "completed"]
print(f"Status enum: {_format_enum_union(status_enum)}")

# Mixed types including null
mixed_enum = ["auto", "manual", None]
print(f"Mixed enum: {_format_enum_union(mixed_enum)}")

# Numeric enum
numeric_enum = [0, 1, 2, 3]
print(f"Numeric enum: {_format_enum_union(numeric_enum)}")

Status enum: "pending" | "active" | "completed"
Mixed enum: "auto" | "manual" | null
Numeric enum: 0 | 1 | 2 | 3


In [5]:
# Schema with enum field
enum_schema = {
    "type": "object",
    "properties": {
        "status": {
            "enum": ["pending", "in_progress", "completed", "failed"],
            "description": "Task status",
        },
        "priority": {"enum": ["p0", "p1", "p2", "p3"], "description": "Priority level"},
    },
    "required": ["status"],
}

print(typescript_schema(enum_schema))

status: "pending" | "in_progress" | "completed" | "failed" - Task status
priority?: "p0" | "p1" | "p2" | "p3" - Priority level


## 4. Default Values

Default values are displayed inline for clarity.

In [6]:
defaults_schema = {
    "type": "object",
    "properties": {
        "verbose": {"type": "boolean", "default": False, "description": "Enable verbose output"},
        "limit": {"type": "integer", "default": 10, "description": "Maximum results"},
        "mode": {"type": "string", "default": "auto", "description": "Processing mode"},
        "threshold": {"type": "number", "default": 0.7, "description": "Similarity threshold"},
        "tags": {"type": "array", "default": None, "description": "Optional tags"},
    },
    "required": [],
}

print(typescript_schema(defaults_schema))

verbose?: bool = false - Enable verbose output
limit?: int = 10 - Maximum results
mode?: string = "auto" - Processing mode
threshold?: float = 0.7 - Similarity threshold
tags?: array = null - Optional tags


## 5. Array Types

Arrays use TypeScript notation with proper item type inference.

In [7]:
array_schema = {
    "type": "object",
    "properties": {
        "tags": {"type": "array", "items": {"type": "string"}, "description": "List of tags"},
        "scores": {"type": "array", "items": {"type": "number"}, "description": "Numeric scores"},
        "entity_ids": {"type": "array", "items": {"type": "string"}, "description": "Entity UUIDs"},
        "flags": {"type": "array", "items": {"type": "boolean"}, "description": "Boolean flags"},
        "any_items": {"type": "array", "description": "Array without item type"},
    },
    "required": ["tags"],
}

print(typescript_schema(array_schema))

tags: string[] - List of tags
scores?: float[] - Numeric scores
entity_ids?: string[] - Entity UUIDs
flags?: bool[] - Boolean flags
any_items?: any[] - Array without item type


## 6. Union Types (anyOf)

The `anyOf` keyword creates union types with `|` separator.

In [8]:
union_schema = {
    "type": "object",
    "properties": {
        "query": {
            "anyOf": [{"type": "string"}, {"type": "array", "items": {"type": "string"}}],
            "description": "Search query or list of queries",
        },
        "limit": {
            "anyOf": [{"type": "integer"}, {"type": "null"}],
            "description": "Maximum results or null for unlimited",
        },
        "confidence": {
            "anyOf": [{"type": "number"}, {"enum": ["auto", "strict"]}],
            "description": "Confidence threshold or mode",
        },
    },
    "required": ["query"],
}

print(typescript_schema(union_schema))

query: string | string[] - Search query or list of queries
limit?: int | null - Maximum results or null for unlimited
confidence?: float | "auto" | "strict" - Confidence threshold or mode


## 7. Nested Objects with Indentation

The `indent` parameter controls nesting levels for nested objects.

In [9]:
# Base level (indent=0)
base_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Name"},
        "value": {"type": "integer", "description": "Value"},
    },
    "required": ["name"],
}

print("Indent level 0:")
print(typescript_schema(base_schema, indent=0))
print("\nIndent level 1:")
print(typescript_schema(base_schema, indent=1))
print("\nIndent level 2:")
print(typescript_schema(base_schema, indent=2))

Indent level 0:
name: string - Name
value?: int - Value

Indent level 1:
  name: string - Name
  value?: int - Value

Indent level 2:
    name: string - Name
    value?: int - Value


## 8. Complete Real-World Examples

Examples from actual khivemcp tool schemas.

In [10]:
# Example 1: Memory recall schema
recall_schema = {
    "type": "object",
    "properties": {
        "action": {
            "enum": ["search", "by_entity", "entities_in_memory"],
            "default": "search",
            "description": "Retrieval strategy",
        },
        "query": {
            "type": "string",
            "description": "search: NL query | by_entity: entity UUID | entities_in_memory: memory UUID",
        },
        "memory_type": {
            "anyOf": [{"type": "string"}, {"type": "null"}],
            "default": None,
            "description": "episodic|semantic|working|null",
        },
        "limit": {"type": "integer", "default": 10, "description": "Max∈[1,100]"},
        "verbose": {
            "type": "boolean",
            "default": False,
            "description": "Include IDs, references, timestamps",
        },
    },
    "required": ["query"],
}

print("Memory Recall Schema:")
print(typescript_schema(recall_schema))

Memory Recall Schema:
action?: "search" | "by_entity" | "entities_in_memory" = "search" - Retrieval strategy
query: string - search: NL query | by_entity: entity UUID | entities_in_memory: memory UUID
memory_type?: string | null = null - episodic|semantic|working|null
limit?: int = 10 - Max∈[1,100]
verbose?: bool = false - Include IDs, references, timestamps


In [11]:
# Example 2: Entity creation schema
entity_schema = {
    "type": "object",
    "properties": {
        "action": {
            "enum": ["create", "get", "update", "delete", "list", "merge"],
            "description": "CRUD operation",
        },
        "entity_id": {
            "anyOf": [{"type": "string"}, {"type": "null"}],
            "default": None,
            "description": "UUID (get/update/delete)",
        },
        "name": {
            "anyOf": [{"type": "string"}, {"type": "null"}],
            "default": None,
            "description": "Name (create)",
        },
        "entity_type": {
            "anyOf": [{"type": "string"}, {"type": "null"}],
            "default": None,
            "description": "Type (create)",
        },
        "properties": {
            "anyOf": [{"type": "object"}, {"type": "null"}],
            "default": None,
            "description": "Dict (create/update, merged)",
        },
        "offset": {"type": "integer", "default": 0, "description": "Skip N"},
        "limit": {"type": "integer", "default": 50, "description": "Max∈[1,100]"},
        "verbose": {
            "type": "boolean",
            "default": False,
            "description": "Include IDs, timestamps, properties",
        },
    },
    "required": ["action"],
}

print("Entity CRUD Schema:")
print(typescript_schema(entity_schema))

Entity CRUD Schema:
action: "create" | "get" | "update" | "delete" | "list" | "merge" - CRUD operation
entity_id?: string | null = null - UUID (get/update/delete)
name?: string | null = null - Name (create)
entity_type?: string | null = null - Type (create)
properties?: object | null = null - Dict (create/update, merged)
offset?: int = 0 - Skip N
limit?: int = 50 - Max∈[1,100]
verbose?: bool = false - Include IDs, timestamps, properties


In [12]:
# Example 3: Complex nested schema with references
complex_schema = {
    "type": "object",
    "properties": {
        "content": {"type": "string", "description": "Memory content"},
        "options": {
            "anyOf": [
                {"$ref": "#/definitions/EpisodicOption"},
                {"$ref": "#/definitions/SemanticOption"},
            ],
            "description": "Memory type-specific options",
        },
        "tags": {"type": "array", "items": {"type": "string"}, "description": "Optional tags"},
        "importance": {"type": "number", "default": 0.5, "description": "Importance score ∈[0,1]"},
        "auto_link": {"type": "boolean", "default": True, "description": "Auto-extract entities"},
    },
    "required": ["content", "options"],
}

print("Complex Nested Schema:")
print(typescript_schema(complex_schema))

Complex Nested Schema:
content: string - Memory content
options: EpisodicOption | SemanticOption - Memory type-specific options
tags?: string[] - Optional tags
importance?: float = 0.5 - Importance score ∈[0,1]
auto_link?: bool = true - Auto-extract entities


## Summary Checklist

**TypeScript Schema Handler Features:**
- ✅ Maps JSON Schema types to TypeScript-like notation (`int`, `float`, `bool`, `string`)
- ✅ Formats enums as literal unions (`"a" | "b" | "c"`)
- ✅ Marks optional fields with `?` suffix
- ✅ Displays default values inline
- ✅ Includes field descriptions for documentation
- ✅ Handles arrays with proper type notation (`type[]`)
- ✅ Supports union types via `anyOf` (`type1 | type2`)
- ✅ Handles `$ref` references to definitions
- ✅ Supports nested objects with indentation
- ✅ Optimized for LLM comprehension (used in khivemcp MCP tools)

**Usage in Lionherd:**
- Tool schema documentation (MCP integration)
- API documentation generation
- Agent-to-agent schema communication
- LLM prompt engineering (clear type constraints)

**Next Steps:**
- See `schema_handlers._json` for JSON Schema generation
- See `schema_handlers._markdown` for markdown table format
- See khivemcp for production usage examples